In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, ROCIndicator, StochasticOscillator
from ta.trend import MACD, SMAIndicator, EMAIndicator, ADXIndicator, PSARIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import VolumeWeightedAveragePrice, OnBalanceVolumeIndicator, AccDistIndexIndicator
from datetime import datetime, timedelta
from sector_mapping import sector_stocks  # Replace with your stock list
from my_stocks import my_stocks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Custom PSAR to avoid warnings
class FixedPSARIndicator(PSARIndicator):
    def __init__(self, high, low, close, step=0.02, max_step=0.2):
        super().__init__(high=high, low=low, close=close, step=step, max_step=max_step)
        
    def _calc_psar(self):
        psar = self._close.copy()
        psar.iloc[:] = np.nan
        
        for i in range(2, len(self._close)):
            if i == 2:
                if self._close.iloc[i] > self._close.iloc[i-1]:
                    trend = 1
                    psar.iloc[i] = min(self._low.iloc[i-1], self._low.iloc[i-2])
                else:
                    trend = -1
                    psar.iloc[i] = max(self._high.iloc[i-1], self._high.iloc[i-2])
                continue
                
            # Rest of original logic with .iloc
            prev_psar = psar.iloc[i-1]
            if trend == 1:
                psar.iloc[i] = prev_psar + self._step * (self._high.iloc[i-1] - prev_psar)
                psar.iloc[i] = min(psar.iloc[i], self._low.iloc[i-1], self._low.iloc[i-2])
            else:
                psar.iloc[i] = prev_psar + self._step * (self._low.iloc[i-1] - prev_psar)
                psar.iloc[i] = max(psar.iloc[i], self._high.iloc[i-1], self._high.iloc[i-2])
            
        return psar

def calculate_signals(ticker, current_date, short_period=14, medium_period=26, long_period=50):
    """Calculate momentum, volume, and trend signals across three timeframes with PSAR"""
    try:
        # Download data - extended for long-term indicators
        end_date = current_date
        start_date = end_date - timedelta(days=long_period*3)
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date, interval='1d')
        
        if df.empty or len(df) < long_period:
            return None

        # Initialize indicators dictionary
        indicators = {}
        
        # Short-term indicators (14 days)
        indicators['short'] = {
            'RSI': RSIIndicator(close=df['Close'], window=short_period).rsi(),
            'ROC': ROCIndicator(close=df['Close'], window=short_period).roc(),
            'Stoch_%K': StochasticOscillator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                window=short_period,
                smooth_window=3
            ).stoch(),
            'SMA': SMAIndicator(close=df['Close'], window=short_period).sma_indicator(),
            'EMA': EMAIndicator(close=df['Close'], window=short_period).ema_indicator(),
            'ATR': AverageTrueRange(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                window=short_period
            ).average_true_range(),
            'PSAR': FixedPSARIndicator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                step=0.02,  # Standard acceleration
                max_step=0.2
            ).psar()
        }
        
        # Medium-term indicators (26 days)
        indicators['medium'] = {
            'RSI': RSIIndicator(close=df['Close'], window=medium_period).rsi(),
            'ROC': ROCIndicator(close=df['Close'], window=medium_period).roc(),
            'SMA': SMAIndicator(close=df['Close'], window=medium_period).sma_indicator(),
            'EMA': EMAIndicator(close=df['Close'], window=medium_period).ema_indicator(),
            'MACD': MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9).macd(),
            'MACD_diff': MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9).macd_diff(),
            'ADX': ADXIndicator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                window=medium_period
            ).adx(),
            'PSAR': FixedPSARIndicator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                step=0.015,  # Slightly slower
                max_step=0.15
            ).psar()
        }
        
        # Long-term indicators (50 days)
        indicators['long'] = {
            'SMA': SMAIndicator(close=df['Close'], window=long_period).sma_indicator(),
            'EMA': EMAIndicator(close=df['Close'], window=long_period).ema_indicator(),
            'BB': BollingerBands(close=df['Close'], window=long_period, window_dev=2),
            'VWAP': VolumeWeightedAveragePrice(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                volume=df['Volume'],
                window=long_period
            ).volume_weighted_average_price(),
            'PSAR': FixedPSARIndicator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                step=0.01,  # Slow acceleration for long-term
                max_step=0.1
            ).psar()
        }
        
        # Volume indicators
        volume_indicators = {
            'OBV': OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume']).on_balance_volume(),
            'ADI': AccDistIndexIndicator(
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                volume=df['Volume']
            ).acc_dist_index(),
            'Volume_MA': df['Volume'].rolling(20).mean(),
            'Volume_Spike': df['Volume'] > (df['Volume'].rolling(20).mean() * 2)
        }
        
        # Get latest values
        latest = {
            'price': df['Close'].iloc[-1],
            'short': {k: v.iloc[-1] for k, v in indicators['short'].items()},
            'medium': {k: v.iloc[-1] if not isinstance(v, dict) else {sk: sv.iloc[-1] for sk, sv in v.items()} 
                      for k, v in indicators['medium'].items()},
            'long': {k: v.iloc[-1] if not hasattr(v, 'bollinger_hband') else {
                'upper': v.bollinger_hband().iloc[-1],
                'middle': v.bollinger_mavg().iloc[-1],
                'lower': v.bollinger_lband().iloc[-1],
                'percent': (df['Close'].iloc[-1] - v.bollinger_lband().iloc[-1]) / 
                          (v.bollinger_hband().iloc[-1] - v.bollinger_lband().iloc[-1])
            } for k, v in indicators['long'].items()},
            'volume': {k: v.iloc[-1] for k, v in volume_indicators.items()}
        }
        
        # Previous values for trend analysis
        prev = {
            'short': {k: v.iloc[-2] for k, v in indicators['short'].items()},
            'medium': {k: v.iloc[-2] if not isinstance(v, dict) else {sk: sv.iloc[-2] for sk, sv in v.items()} 
                     for k, v in indicators['medium'].items()},
        }
        
        # Price changes
        price_changes = {
            'short': (latest['price'] - df['Close'].iloc[-short_period]) / df['Close'].iloc[-short_period] * 100,
            'medium': (latest['price'] - df['Close'].iloc[-medium_period]) / df['Close'].iloc[-medium_period] * 100,
            'long': (latest['price'] - df['Close'].iloc[-long_period]) / df['Close'].iloc[-long_period] * 100
        }
        
        # Trend analysis with PSAR
        trends = {
            # Moving average trends
            'short_term_up': latest['price'] > latest['short']['SMA'] > latest['short']['EMA'],
            'medium_term_up': latest['price'] > latest['medium']['SMA'] > latest['medium']['EMA'],
            'long_term_up': latest['price'] > latest['long']['SMA'] > latest['long']['EMA'],
            'golden_cross': latest['short']['SMA'] > latest['medium']['SMA'] and prev['short']['SMA'] <= prev['medium']['SMA'],
            'death_cross': latest['short']['SMA'] < latest['medium']['SMA'] and prev['short']['SMA'] >= prev['medium']['SMA'],
            'macd_bullish': latest['medium']['MACD_diff'] > 0 and prev['medium']['MACD_diff'] <= 0,
            'macd_bearish': latest['medium']['MACD_diff'] < 0 and prev['medium']['MACD_diff'] >= 0,
            'adx_strength': latest['medium']['ADX'] > 25,
            
            # PSAR trends
            'sar_short_bullish': latest['price'] > latest['short']['PSAR'],
            'sar_medium_bullish': latest['price'] > latest['medium']['PSAR'],
            'sar_long_bullish': latest['price'] > latest['long']['PSAR'],
            'sar_short_bearish': latest['price'] < latest['short']['PSAR'],
            'sar_medium_bearish': latest['price'] < latest['medium']['PSAR'],
            'sar_long_bearish': latest['price'] < latest['long']['PSAR'],
        }
        
        # Momentum signals
        momentum = {
            'rsi_short': latest['short']['RSI'],
            'rsi_medium': latest['medium']['RSI'],
            'stoch_overbought': latest['short']['Stoch_%K'] > 80,
            'stoch_oversold': latest['short']['Stoch_%K'] < 20,
            'roc_short': latest['short']['ROC'],
            'roc_medium': latest['medium']['ROC'],
            'bb_position': latest['long']['BB']['percent'],
            'atr': latest['short']['ATR']
        }
        
        # Volume signals
        volume = {
            'obv_trend': '↑' if latest['volume']['OBV'] > volume_indicators['OBV'].iloc[-2] else '↓',
            'adi_trend': '↑' if latest['volume']['ADI'] > volume_indicators['ADI'].iloc[-2] else '↓',
            'volume_spike': latest['volume']['Volume_Spike'],
            'vwap_relation': 'above' if latest['price'] > latest['long']['VWAP'] else 'below'
        }
        
        # Generate signal reasons
        signal_reasons = []
        
        # Trend reasons
        if trends['golden_cross']:
            signal_reasons.append("Golden Cross (Short > Medium MA)")
        if trends['death_cross']:
            signal_reasons.append("Death Cross (Short < Medium MA)")
        if trends['sar_short_bullish'] and trends['sar_medium_bullish'] and trends['sar_long_bullish']:
            signal_reasons.append("PSAR Bullish (All Timeframes)")
        elif trends['sar_short_bullish'] or trends['sar_medium_bullish']:
            signal_reasons.append("PSAR Bullish (Partial)")
        if trends['adx_strength']:
            signal_reasons.append("Strong Trend (ADX > 25)")
            
        # Momentum reasons
        if momentum['rsi_short'] > 70:
            signal_reasons.append(f"Short-term RSI Overbought ({momentum['rsi_short']:.1f})")
        if momentum['rsi_short'] < 30:
            signal_reasons.append(f"Short-term RSI Oversold ({momentum['rsi_short']:.1f})")
        if momentum['stoch_overbought']:
            signal_reasons.append("Stochastic Overbought")
        if momentum['stoch_oversold']:
            signal_reasons.append("Stochastic Oversold")
        if momentum['bb_position'] > 0.8:
            signal_reasons.append(f"BB Upper Band ({momentum['bb_position']:.2%})")
        if momentum['bb_position'] < 0.2:
            signal_reasons.append(f"BB Lower Band ({momentum['bb_position']:.2%})")
            
        # Volume reasons
        if volume['volume_spike']:
            signal_reasons.append("Volume Spike (2x MA)")
        if volume['obv_trend'] == '↑' and volume['volume_spike']:
            signal_reasons.append("OBV Uptick with Volume")
        if volume['vwap_relation'] == 'above':
            signal_reasons.append("Price Above VWAP")
            
        # Price change reasons
        if price_changes['short'] > 10:
            signal_reasons.append(f"Short-term Up {price_changes['short']:.1f}%")
        if price_changes['short'] < -5:
            signal_reasons.append(f"Short-term Down {abs(price_changes['short']):.1f}%")
        
        # Generate composite score (0-100)
        score = 50  # Neutral starting point
        
        # Trend factors (30%)
        score += 5 if trends['short_term_up'] else -10
        score += 5 if trends['medium_term_up'] else -5
        score += 5 if trends['long_term_up'] else -3
        score += 5 if trends['golden_cross'] else (-10 if trends['death_cross'] else 0)
        score += 5 if trends['macd_bullish'] else (-5 if trends['macd_bearish'] else 0)
        score += 5 if trends['adx_strength'] else 0
        
        # PSAR factors (15%)
        score += 5 if trends['sar_short_bullish'] else -5
        score += 5 if trends['sar_medium_bullish'] else -5
        score += 5 if trends['sar_long_bullish'] else -5
        
        # Momentum factors (25%)
        score += 5 * ((momentum['rsi_short'] - 50) / 10)  # Normalized RSI contribution
        score += 5 if momentum['stoch_oversold'] else (-5 if momentum['stoch_overbought'] else 0)
        score += 5 * (momentum['roc_short'] / 5)  # Normalized ROC contribution
        score += 5 * (momentum['roc_medium'] / 3)  # Normalized ROC contribution
        score += 5 * (momentum['bb_position'] - 0.5)  # BB position contribution
        
        # Volume factors (20%)
        score += 10 if volume['obv_trend'] == '↑' else -10
        score += 5 if volume['adi_trend'] == '↑' else -3
        score += 5 if volume['volume_spike'] and volume['obv_trend'] == '↑' else (
                 -5 if volume['volume_spike'] and volume['obv_trend'] == '↓' else 0)
        score += 5 if volume['vwap_relation'] == 'above' else -3
        
        # Price action factors (10%)
        score += 5 * (price_changes['short'] / 5)  # Normalized short-term change
        score += 3 * (price_changes['medium'] / 3)  # Normalized medium-term change
        score += 2 * (price_changes['long'] / 2)  # Normalized long-term change
        
        # Cap score between 0 and 100
        score = max(0, min(100, score))
        
        # Generate trading signal
        signal = "HOLD"
        signal_strength = ""
        
        # STRONG BUY: All PSAR timeframes bullish + high score
        if (score > 80 and 
            trends['sar_short_bullish'] and 
            trends['sar_medium_bullish'] and 
            trends['sar_long_bullish'] and 
            volume['obv_trend'] == '↑'):
            signal = "STRONG BUY"
            signal_strength = "(Multi-Timeframe PSAR Confirmed)"
        
        # BUY: Partial PSAR confirmation
        elif (score > 65 and 
              (trends['sar_short_bullish'] or trends['sar_medium_bullish']) and 
              volume['obv_trend'] == '↑'):
            signal = "BUY"
            signal_strength = "(PSAR Bullish)"
            
        # STRONG SELL: All PSAR timeframes bearish + low score
        elif (score < 20 and 
              trends['sar_short_bearish'] and 
              trends['sar_medium_bearish'] and 
              trends['sar_long_bearish'] and 
              volume['obv_trend'] == '↓'):
            signal = "STRONG SELL"
            signal_strength = "(Multi-Timeframe PSAR Confirmed)"
            
        # SELL: Partial PSAR confirmation
        elif (score < 35 and 
              (trends['sar_short_bearish'] or trends['sar_medium_bearish']) and 
              volume['obv_trend'] == '↓'):
            signal = "SELL"
            signal_strength = "(PSAR Bearish)"
        
        # Add reasons if not already included
        if signal != "HOLD" and signal_reasons:
            signal += f" {signal_strength} [{', '.join(signal_reasons)}]"
        elif signal == "HOLD" and signal_reasons:
            signal = f"HOLD (Conflicting signals: {', '.join(signal_reasons)})"
            
        # Prepare result dictionary
        result = {
            'Ticker': ticker,
            'Price': f"{latest['price']:.2f}",
            f'Change_{short_period}D': f"{price_changes['short']:.1f}%",
            f'Change_{medium_period}D': f"{price_changes['medium']:.1f}%",
            f'Change_{long_period}D': f"{price_changes['long']:.1f}%",
            f'RSI_{short_period}': f"{momentum['rsi_short']:.1f}",
            f'RSI_{medium_period}': f"{momentum['rsi_medium']:.1f}",
            f'Stoch_%K_{short_period}': f"{latest['short']['Stoch_%K']:.1f}",
            f'MACD_diff_{medium_period}': f"{latest['medium']['MACD_diff']:.3f}",
            'BB_%': f"{momentum['bb_position']:.2%}",
            f'SMA_{short_period}/{medium_period}/{long_period}': f"{latest['short']['SMA']:.1f}/{latest['medium']['SMA']:.1f}/{latest['long']['SMA']:.1f}",
            f'PSAR_{short_period}': 'Bullish' if trends['sar_short_bullish'] else 'Bearish',
            f'PSAR_{medium_period}': 'Bullish' if trends['sar_medium_bullish'] else 'Bearish',
            f'PSAR_{long_period}': 'Bullish' if trends['sar_long_bullish'] else 'Bearish',
            'Volume': f"{df['Volume'].iloc[-1]/1e6:.1f}M",
            'Volume_Spike': 'Yes' if volume['volume_spike'] else 'No',
            'OBV_Trend': volume['obv_trend'],
            'VWAP_Relation': volume['vwap_relation'],
            'Score': f"{score:.1f}",
            'Signal': signal
        }
        
        return result
        
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None


In [13]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

class StrategyBacktester:
    def __init__(self, capital=1_000_000, top_n=5):
        self.capital = capital
        self.top_n = top_n
        self.portfolio = {'cash': capital, 'holdings': {}, 'history': []}
        self.price_cache = {}  # Cache to store recent prices
        
    def run_backtest(self, start_date, end_date, stock_universe):
        current_date = start_date
        while current_date <= end_date:
            if current_date.weekday() == 0:  # Run weekly on Mondays
                self.run_weekly_rebalance(current_date, stock_universe)
                print(f"Testing for date {current_date}")
                print(f"")
            current_date += timedelta(days=1)
        
        self.generate_report()
    
    def run_weekly_rebalance(self, current_date, stock_universe):
        # Get signals for all stocks
        signals = []
        for ticker in stock_universe:
            try:
                signal = calculate_signals(ticker, current_date)
                if signal:
                    signals.append(signal)
            except Exception as e:
                print(f"Error processing {ticker}: {str(e)}")
        
        # Sort by score and pick top N
        signals.sort(key=lambda x: float(x['Score']), reverse=True)
        top_stocks = signals[:self.top_n]
        
        # Calculate position size
        position_size = self.capital / len(top_stocks) if top_stocks else 0
        
        # Close positions not in top stocks
        for ticker in list(self.portfolio['holdings'].keys()):
            if ticker not in [s['Ticker'] for s in top_stocks]:
                self.close_position(ticker, current_date)
        
        # Open new positions
        for stock in top_stocks:
            ticker = stock['Ticker']
            current_price = float(stock['Price'])
            
            if ticker not in self.portfolio['holdings']:
                if stock['Signal'].startswith(('BUY', 'STRONG BUY')):
                    shares = position_size / current_price
                    self.open_position(ticker, shares, current_price, current_date)
            
            # Handle existing positions
            else:
                if stock['Signal'].startswith(('SELL', 'STRONG SELL')):
                    self.close_position(ticker, current_date)
        
        # Record portfolio snapshot
        self.record_portfolio(current_date)
    
    def get_current_price(self, ticker, date):
        """Safe method to get current price with fallback"""
        try:
            # Check cache first
            if ticker in self.price_cache and self.price_cache[ticker]['date'] == date:
                return self.price_cache[ticker]['price']
            
            # Get fresh data
            price_data = yf.Ticker(ticker).history(
                start=date - timedelta(days=5),
                end=date + timedelta(days=1))
            
            if not price_data.empty:
                price = price_data['Close'].iloc[-1]
                self.price_cache[ticker] = {'date': date, 'price': price}
                return price
            return None
        except:
            return None
    
    def open_position(self, ticker, shares, price, date):
        cost = shares * price
        if cost > self.portfolio['cash']:
            shares = self.portfolio['cash'] / price
            cost = shares * price
        
        self.portfolio['holdings'][ticker] = {
            'shares': shares,
            'entry_price': price,
            'entry_date': date,
            'last_price': price  # Initialize last known price
        }
        self.portfolio['cash'] -= cost
    
    def close_position(self, ticker, date):
        if ticker in self.portfolio['holdings']:
            position = self.portfolio['holdings'][ticker]
            current_price = self.get_current_price(ticker, date)
            
            # Use last known price if current unavailable
            if current_price is None:
                current_price = position['last_price']
                print(f"Using last known price for {ticker} on {date}")
            
            proceeds = position['shares'] * current_price
            self.portfolio['cash'] += proceeds
            del self.portfolio['holdings'][ticker]
    
    def record_portfolio(self, date):
        total_value = self.portfolio['cash']
        for ticker, position in self.portfolio['holdings'].items():
            current_price = self.get_current_price(ticker, date)
            
            if current_price is None:
                current_price = position['last_price']
            else:
                # Update last known price
                self.portfolio['holdings'][ticker]['last_price'] = current_price
            
            total_value += position['shares'] * current_price
        
        self.portfolio['history'].append({
            'date': date,
            'value': total_value,
            'holdings': {k: v['shares'] for k,v in self.portfolio['holdings'].items()}
        })
    
    def generate_report(self):
        # Create performance dataframe
        df = pd.DataFrame(self.portfolio['history'])
        df.set_index('date', inplace=True)
        
        # Calculate metrics
        initial_value = self.capital
        final_value = df['value'].iloc[-1]
        total_return = (final_value - initial_value) / initial_value * 100
        max_drawdown = (df['value'].cummax() - df['value']).max() / df['value'].cummax().max() * 100
        
        # Plot performance
        plt.figure(figsize=(12, 6))
        plt.plot(df.index, df['value'], label='Portfolio Value')
        plt.title(f'Strategy Performance\nTotal Return: {total_return:.2f}% | Max Drawdown: {max_drawdown:.2f}%')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value ($)')
        plt.legend()
        plt.grid()
        plt.savefig('backtest_results_17_06.png')
        plt.close()
        
        # Generate trade log
        trade_log = []
        for date, holdings in zip(df.index, df['holdings']):
            for ticker in holdings:
                if ticker in self.portfolio['holdings']:
                    position = self.portfolio['holdings'][ticker]
                    trade_log.append({
                        'ticker': ticker,
                        'entry_date': position['entry_date'],
                        'exit_date': date,
                        'return_pct': (position['last_price'] - position['entry_price']) / position['entry_price'] * 100
                    })
        
        trade_df = pd.DataFrame(trade_log)
        if not trade_df.empty:
            trade_df.to_csv('trade_log_17_06.csv', index=False)
        
        print(f"Backtest complete. Final portfolio value: ${final_value:,.2f}")
        print(f"Total return: {total_return:.2f}%")
        print(f"Max drawdown: {max_drawdown:.2f}%")

# Example usage
if __name__ == "__main__":
    # Configuration
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2024, 12, 31)
    stock_universe = ["BAJAJINDEF.NS","SCI.NS","NRAIL.NS","JUBLINGREA.NS","GUJAPOLLO.NS","DBREALTY.NS","MANCREDIT.NS","OSWALAGRO.NS",
                     "NAZARA.NS","GOODLUCK.NS","INDOTHAI.NS","KERNEX.NS","DELPHIFX.NS","KDDL.NS","SUBROS.NS"]
    
    # Run backtest
    backtester = StrategyBacktester(capital=1_000_000, top_n=5)
    backtester.run_backtest(start_date, end_date, stock_universe)

$BAJAJINDEF.NS: possibly delisted; no price data found  (1d 2023-08-04 00:00:00 -> 2024-01-01 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1691087400, endDate = 1704047400")
$MANCREDIT.NS: possibly delisted; no price data found  (1d 2023-08-04 00:00:00 -> 2024-01-01 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1691087400, endDate = 1704047400")


Testing for date 2024-01-01 00:00:00



$BAJAJINDEF.NS: possibly delisted; no price data found  (1d 2023-08-11 00:00:00 -> 2024-01-08 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1691692200, endDate = 1704652200")
$MANCREDIT.NS: possibly delisted; no price data found  (1d 2023-08-11 00:00:00 -> 2024-01-08 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1691692200, endDate = 1704652200")


Testing for date 2024-01-08 00:00:00



$BAJAJINDEF.NS: possibly delisted; no price data found  (1d 2023-08-18 00:00:00 -> 2024-01-15 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1692297000, endDate = 1705257000")
$MANCREDIT.NS: possibly delisted; no price data found  (1d 2023-08-18 00:00:00 -> 2024-01-15 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1692297000, endDate = 1705257000")


...

Testing for date 2024-12-16 00:00:00



$BAJAJINDEF.NS: possibly delisted; no price data found  (1d 2024-07-26 00:00:00 -> 2024-12-23 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1721932200, endDate = 1734892200")


Testing for date 2024-12-23 00:00:00



$BAJAJINDEF.NS: possibly delisted; no price data found  (1d 2024-08-02 00:00:00 -> 2024-12-30 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1722537000, endDate = 1735497000")


Testing for date 2024-12-30 00:00:00

Backtest complete. Final portfolio value: $1,462,147.69
Total return: 46.21%
Max drawdown: 17.63%


In [14]:
backtester.generate_report()

Backtest complete. Final portfolio value: $1,462,147.69
Total return: 46.21%
Max drawdown: 17.63%


In [9]:
df = pd.DataFrame(backtester.portfolio['history'])
df.set_index('date', inplace=True)

# Calculate metrics
initial_value = backtester.capital
final_value = df['value'].iloc[-1]
total_return = (final_value - initial_value) / initial_value * 100
max_drawdown = (df['value'].cummax() - df['value']).max() / df['value'].cummax().max() * 100

# Plot performance
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value'], label='Portfolio Value')
plt.title(f'Strategy Performance\nTotal Return: {total_return:.2f}% | Max Drawdown: {max_drawdown:.2f}%')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid()
plt.savefig('backtest_results.png')
plt.close()

In [12]:
df

,value,holdings
date,,
2023-01-02,1.000005e+06,{'JPM': 1590.4572564612326}
2023-01-09,1.003821e+06,"{'JPM': 1590.4572564612326, 'META': 1545.47562..."
2023-01-16,1.028453e+06,"{'JPM': 1590.4572564612326, 'META': 1545.47562..."
2023-01-23,1.060083e+06,"{'META': 1545.4756201220926, 'V': 935.97903406..."
2023-01-30,1.061273e+06,"{'META': 1545.4756201220926, 'AMZN': 2038.3204..."
2023-02-06,1.164035e+06,"{'META': 1545.4756201220926, 'AMZN': 2038.3204..."
2023-02-13,1.134493e+06,"{'META': 1545.4756201220926, 'AAPL': 1388.3104..."
2023-02-20,1.137027e+06,"{'META': 1545.4756201220926, 'AAPL': 1388.3104..."
2023-02-27,1.124706e+06,"{'META': 1545.4756201220926, 'AAPL': 1388.3104..."


In [16]:
backtester.portfolio['holdings']

{'KERNEX.NS': {'shares': 137.72681885480148,
  'entry_price': 1452.15,
  'entry_date': datetime.datetime(2024, 12, 16, 0, 0),
  'last_price': 1281.75},
 'NRAIL.NS': {'shares': 534.4735435595938,
  'entry_price': 374.2,
  'entry_date': datetime.datetime(2024, 12, 30, 0, 0),
  'last_price': 360.20001220703125},
 'INDOTHAI.NS': {'shares': 172.94305849798954,
  'entry_price': 1156.45,
  'entry_date': datetime.datetime(2024, 12, 30, 0, 0),
  'last_price': 1214.25},
 'MANCREDIT.NS': {'shares': 1333.2444503699753,
  'entry_price': 150.01,
  'entry_date': datetime.datetime(2024, 12, 30, 0, 0),
  'last_price': 151.3000030517578}}